### BW64 - 진정한 병렬성을 살리려면 concurrent.futures를 사용하라.

- 파이썬의 프로그램을 작성하다 보면 성능의 벽이 느껴진다. <br/> 코드를 최적한 후에도 프로그램 실행 속도가 필요한 수준보다 더 느릴 것이다. <br/> CPU의 코어 수가 늘어난
현대의 컴퓨터에서는 병렬성이 한 가지 해결책이라고 봐도 무방하다.

- 불행하게 파이썬의 전역 인터프리터 잠금(GLE)이 스레드에서 진정한 병렬성을 막기에 이옵션은 사용불가하다.

> 인터프리터 잠금(GLE) ?
- 간단하게 말하면 한번에 하나의 스레드만 수행할 수 있게 락을 것는 것이다.

- 다른 일반적인 조건은 C언어를 사용한 확장 모듈을 사용하는 것이다.
<br/> 이유는 파이썬의 연산 처리 속도가 C에 비해서 느리기에 빠른 연산을 요구하는 작업은 C언어가 좋다.
<br/> GLE를 신경쓰지 않고 여러 CPU 코어를 활용함이 가능함과 동시에 C확장 API문서화가 잘되어 있다.
<br/><br/>
- 하지만 C로 재작성 하려면 돈이 많이 들고 <br/> 파이썬을 C로 실행 가능하게 하려면 원 코드와 똑같이 작동 하는지 엄청난 테스트가 필요며 위험이 따른다는 단점이 있다.
<br/> 그렇기에 어려운 계산 문제를 해결하기 위한 더 좋은 방법이 있다.
<br/><br/>
- concurrent.futures로 쉽게 접근할 수 있는 내장 모듈 multiprocessing이 바로 이 상황에 필요하다.
<br/> 이 모듈을 사용하면 파이썬에서 자식 프로세서로 추가적인 인터프리터를 실행하여 병렬로 여러 CPU코어를 활용가능하다.
<br/> 주 인터프리터의 GIL과 분리되기에 각 자식 프로세서는 CPU코어 하나를 완전히 활용할 수 있다.
<br/><br/> Ex. CPU코어를 활용해 집약적인 계산을 한다고 가정하면 

In [14]:
# mymodule.py
from time import time
from concurrent.futures import ThreadPoolExecutor

def gcd(pair):
    a, b = pair
    low = min(a, b)
    for i in range(low, 0, -1):
        if a % i == 0 and b % i == 0:
            return i
    assert False, '도달할 수 없음'

- 이 함수를 순차적으로 실행하면 병렬성이 없기 때문에 계산 시간이 선형적으로 증가함.

In [24]:
# run_serial.py

import my_module
import time

NUMBERS = [
    (1963309, 2265973), (2030677, 3814172),
    (1551645, 2229620), (2039045, 2020802),
    (1823712, 1924928), (2293129, 1020491),
    (1281238, 2273782), (3823812, 4237281),
    (3812741, 4729139), (1292391, 2123811),
]

def main():
    start = time.time()
    results = list(map(my_module.gcd, NUMBERS))
    end = time.time()
    delta = end - start
    print(f'총 {delta:.3f} 초 걸림')


if __name__ == '__main__':
    main()
    
# >>> 총 0.911초 걸림

ModuleNotFoundError: No module named 'my_module'

- 여러 파이썬 스레드에서 이코드를 실행하면 <br/> GIL 때문에 병렬로 여러 CPU코어를 사용하지 못해서 속도가 개선되지 않는다.
<br/><br/>
Ex. 이번엔 concurrent.futures 모듈의 ThreadPoolExecutor 클래스를 사용하여 진행해봤습니다.

In [25]:
# run_threads.py
import my_module
from concurrent.futures import ThreadPoolExecutor
import time

NUMBERS = [
    (1963309, 2265973), (2030677, 3814172),
    (1551645, 2229620), (2039045, 2020802),
    (1823712, 1924928), (2293129, 1020491),
    (1281238, 2273782), (3823812, 4237281),
    (3812741, 4729139), (1292391, 2123811),
]

def main():
    start = time.time()
    pool = ThreadPoolExecutor(max_workers=2)
    results = list(pool.map(my_module.gcd, NUMBERS))
    end = time.time()
    delta = end - start
    print(f'총 {delta:.3f} 초 걸림')

if __name__ == '__main__':
    main()
    
# >>> 총 1.436초 걸림

ModuleNotFoundError: No module named 'my_module'

- 위의 결과는 스레드 풀을 시작하고 통신하는데 드는 오버헤드 때문에 시간이 증가한다.
<br/><br/> 마지막으로 concurrent.futures모듈에 있는 
ThreadPoolExecutor를 같은 모듈의 ProcessPoolExecutor로 대체해주면 속도가 빨라지는걸 확인할 수 있다.

In [26]:
# run_parallel.py
import my_module
from concurrent.futures import ProcessPoolExecutor
import time

NUMBERS = [
    (1963309, 2265973), (2030677, 3814172),
    (1551645, 2229620), (2039045, 2020802),
    (1823712, 1924928), (2293129, 1020491),
    (1281238, 2273782), (3823812, 4237281),
    (3812741, 4729139), (1292391, 2123811),
]

def main():
    start = time.time()
    pool = ProcessPoolExecutor(max_workers=2) # 이 부분만 바꿈
    results = list(pool.map(my_module.gcd, NUMBERS))
    end = time.time()
    delta = end - start
    print(f'총 {delta:.3f} 초 걸림')

if __name__ == '__main__':
    main()

# >>> 총 0.683초 걸림

ModuleNotFoundError: No module named 'my_module'

- CPU 코어를 사용해서 실행했는데 확실히 빨라진걸 볼 수 있다.
<br/><br/>
- 다음은 processPoolExecutor 클래스가 (multiprocessing 모듈이 제공하는 저수준 요소 활용) 실제로 하는일입니다.
<br/><br/>
- 01. number 입력 데이터에서 map으로 각 아이템을 가져왔고
- 02. pickle 모듈을 사용해서 바이너리 데이터로 직렬화를 하였고
- 03. 주 인터프리터 프로세서에서 직렬화한 데이터를 지역 소켓을 통해 자식 인터프리터 프로세스로 복사해줬고
- 04. 자식 프로세서에서 pickle을 사용해서 데이터를 파이썬 객체로 역직렬화
- 05. gcd함수가 들어 있는 파이썬 모듈을 임포트해주었다.
- 06. 다른 자식 프로세서를 사용해서 병렬로 입력 데이터에 함수를 실행했고
- 07. 결과를 다시 바이트로 직렬화했다.
- 08. 소켓을 통해 바이트를 다시 복사했고
- 09. 바이트를 부모 프로세서에 있는 파이썬 객체로 역직렬화함.
- 10. 마지막으로 여러 자식에 있는 결과를 반환용으로 한개의 리스트를 만든다.
<br/><br/>
- 코드는 간단해 보이지만 multiprocessing 모듈과 processPoolExecutor 클래스는 병렬성을 활용하고자 아주 많은 일을 한다.

- multiprocessing의 비용은 부모와 자식 프로세스 간에 일어날 수 밖에 없는 모든 직렬화와 역직렬화 때문에 상당히 높다.
<br/><br/> 
- 이방식은 서로 잘격리되고 레버리지가 큰 유형의 작업에는 잘 맞는다.
<br/> 여기서 레버리지는 부모 자식사이에 주고 받아야 하는 데이터 크기는 작지만 자식 프로세서가 계산해야하는 양이 많다는 뜻
<br/> 아까보았던 첫번째 예시가 바로 그런예이다. 여러 수학알고리즘도 비슷하게 작동한다.
<br/><br/> 
- 계산이 위에서 말한 특성이 없다면 multiprocessing의 비용이 병렬성을 통한 속도 향상을 막기도 한다.
<br/> 이러한 상황에 쓸수 있는 고급 기능들을 제시하지만 너무 복잡해진다.
<br/> 이 복잡함을 다른 프로세서로 확장하면 훨씬 이해하기가 어렵다.
<br/> 
### 결론은 
- multiprocessing의 모든 기능을 직접 사용하지 말고 더 간단한 concurrent.futures 모듈을 통해 사용하도록 하자.
<br/> ThreadPoolExecutor클래스를 사용해서 천천히 시작하고 나중에 속도를 올리려면 ProcessPoolExecutor로 욺기고
<br/> 그래도 선택지가 없다면 multiprocessing 모듈을 사용하는 방안 고려하자.